In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import sys
import os
sys.path.insert(0, os.path.abspath('../'))
from DNN_module import Net

%run ../NCoinDP_functions.ipynb

In [2]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
if torch.cuda.is_available(): 
    dev = "cuda:0" 
else: 
    dev = "cpu"
device = torch.device(dev) 

2.1.2+cu121
True
NVIDIA A10


In [25]:
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from DNN_module import Net
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

# Default : cuda
torch.set_default_device('cuda')

# Number of train and validation data
L_train = 250000
L_val   =  50000

# Define the batch size
BATCH_SIZE = 64

# Number of Epochs
N_EPOCHS = 300
    
# Number of Simulations
sims = 20

tmp = "infer_sim_0.pt"
tmp = torch.load(tmp)

# Data import
[X, Y] = tmp
X = X.to(device)
Y = Y.to(device)

# Weighted loss function
torch.set_default_device('cuda')
def weighted_mse_loss(input, target, weight):
        return (weight * (input - target) ** 2).sum()
out_range = [torch.min(Y,0).values.detach().cpu().numpy(), torch.max(Y,0).values.detach().cpu().numpy()]

weight_1 = torch.tensor(1/(out_range[1] - out_range[0])**2)

for sim in range(sims):
    
    torch.manual_seed(1000 + sim)
    indexes = torch.randperm(L_train + L_val)

    # Divide Data
    X_train = X[indexes[0:L_train]]
    Y_train = Y[indexes[0:L_train]]

    X_val = X[indexes[L_train:(L_train + L_val)]]
    Y_val = Y[indexes[L_train:(L_train + L_val)]]

    # Use torch.utils.data to create a DataLoader that will take care of creating batches
    dataset = TensorDataset(X_train, Y_train)
    dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True, generator=torch.Generator(device=device))

    # Get the dataset size for printing (it is equal to N_SAMPLES)
    dataset_size = len(dataloader.dataset)

    # Define the input and output dimensions
    D_in, D_out = X_train.size()[1], Y_train.size()[1]

    # Create an instance of the Net class with specified dimensions
    H, H2, H3 = 256, 256, 256
    net = Net(D_in = D_in, D_out = D_out, H = H, H2 = H2, H3 = H3)

    # Model name
    model_save_name = 'infer_nets/net0/'+str(sim)+'.pt'
    path = F"./{model_save_name}"

    # The nn package also contains definitions of popular loss functions; in this case we will use Mean Squared Error (MSE) as our loss function.
    #loss_fn = torch.nn.MSELoss(reduction='sum')
    learning_rate = 1e-5
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)

    train_error_plt = []
    val_error_plt = []

    torch.manual_seed(2000 + sim)

    # Loop over epochs
    for epoch in range(N_EPOCHS):
        for id_batch, (x_batch, y_batch) in enumerate(dataloader):
            y_batch_pred = net(x_batch)
            loss = weighted_mse_loss(y_batch, y_batch_pred, weight_1)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if epoch % 30 ==0 and id_batch % 500 == 0:
                loss, current = loss.item(), (id_batch + 1)* len(x_batch)
                print(f"train_loss: {loss/BATCH_SIZE:>7f}  [{current:>5d}/{dataset_size:>5d}]")

        with torch.no_grad():
            net.eval()
            theta_pred_train = net(X_train)
            train_loss = weighted_mse_loss(Y_train, theta_pred_train, weight_1) / L_train
           
            train_error_plt = np.append(train_error_plt, train_loss.to("cpu"))

            theta_pred_val = net(X_val)
            #val_loss = loss_fn(Y_val, theta_pred_val) / L_val
            val_loss = weighted_mse_loss(Y_val, theta_pred_val, weight_1) / L_val
         
            val_error_plt = np.append(val_error_plt, val_loss.to("cpu"))

        if epoch % 10 ==0:
            print(f"Epoch {epoch + 1}\n-------------------------------")
            print(f"train_loss {train_loss:>7f} val_loss {val_loss:>7f}")
            print("learning rate: ", learning_rate, ", sim: ", sim)

        ## Choose Best Model
        if val_error_plt[epoch] == np.min(val_error_plt):
             best=epoch
             torch.save(net.state_dict(), path)

        if epoch % 100 ==99:
            net.load_state_dict(torch.load(path))
            learning_rate = max(learning_rate * 1e-1, 1e-9)


train_loss: 2.307885  [   64/250000]
train_loss: 0.805660  [32064/250000]
train_loss: 0.356383  [64064/250000]
train_loss: 0.252312  [96064/250000]
train_loss: 0.092206  [128064/250000]
train_loss: 0.092058  [160064/250000]
train_loss: 0.042132  [192064/250000]
train_loss: 0.039139  [224064/250000]
Epoch 1
-------------------------------
train_loss 0.021467 val_loss 0.021419
learning rate:  1e-05 , sim:  0
Epoch 11
-------------------------------
train_loss 0.008743 val_loss 0.008766
learning rate:  1e-05 , sim:  0
Epoch 21
-------------------------------
train_loss 0.009294 val_loss 0.009324
learning rate:  1e-05 , sim:  0
train_loss: 0.007980  [   64/250000]
train_loss: 0.009628  [32064/250000]
train_loss: 0.008950  [64064/250000]
train_loss: 0.010463  [96064/250000]
train_loss: 0.009738  [128064/250000]
train_loss: 0.010244  [160064/250000]
train_loss: 0.008300  [192064/250000]
train_loss: 0.008901  [224064/250000]
Epoch 31
-------------------------------
train_loss 0.008770 val_los

Epoch 71
-------------------------------
train_loss 0.008356 val_loss 0.008442
learning rate:  1e-05 , sim:  1
Epoch 81
-------------------------------
train_loss 0.008307 val_loss 0.008394
learning rate:  1e-05 , sim:  1
train_loss: 0.007863  [   64/250000]
train_loss: 0.008050  [32064/250000]
train_loss: 0.008211  [64064/250000]
train_loss: 0.008258  [96064/250000]
train_loss: 0.008226  [128064/250000]
train_loss: 0.007793  [160064/250000]
train_loss: 0.009242  [192064/250000]
train_loss: 0.007698  [224064/250000]
Epoch 91
-------------------------------
train_loss 0.008331 val_loss 0.008409
learning rate:  1e-05 , sim:  1
Epoch 101
-------------------------------
train_loss 0.008376 val_loss 0.008457
learning rate:  1.0000000000000002e-06 , sim:  1
Epoch 111
-------------------------------
train_loss 0.008296 val_loss 0.008379
learning rate:  1.0000000000000002e-06 , sim:  1
train_loss: 0.008471  [   64/250000]
train_loss: 0.008029  [32064/250000]
train_loss: 0.008303  [64064/250000

train_loss: 0.006926  [   64/250000]
train_loss: 0.009190  [32064/250000]
train_loss: 0.007257  [64064/250000]
train_loss: 0.009671  [96064/250000]
train_loss: 0.009597  [128064/250000]
train_loss: 0.007192  [160064/250000]
train_loss: 0.008970  [192064/250000]
train_loss: 0.008013  [224064/250000]
Epoch 151
-------------------------------
train_loss 0.008277 val_loss 0.008414
learning rate:  1.0000000000000002e-06 , sim:  2
Epoch 161
-------------------------------
train_loss 0.008656 val_loss 0.008793
learning rate:  1.0000000000000002e-06 , sim:  2
Epoch 171
-------------------------------
train_loss 0.008298 val_loss 0.008430
learning rate:  1.0000000000000002e-06 , sim:  2
train_loss: 0.008289  [   64/250000]
train_loss: 0.008040  [32064/250000]
train_loss: 0.007407  [64064/250000]
train_loss: 0.008058  [96064/250000]
train_loss: 0.008539  [128064/250000]
train_loss: 0.008416  [160064/250000]
train_loss: 0.007356  [192064/250000]
train_loss: 0.008545  [224064/250000]
Epoch 181
---

train_loss: 0.010081  [128064/250000]
train_loss: 0.008583  [160064/250000]
train_loss: 0.006872  [192064/250000]
train_loss: 0.006878  [224064/250000]
Epoch 211
-------------------------------
train_loss 0.008730 val_loss 0.008861
learning rate:  1.0000000000000002e-07 , sim:  3
Epoch 221
-------------------------------
train_loss 0.009138 val_loss 0.009281
learning rate:  1.0000000000000002e-07 , sim:  3
Epoch 231
-------------------------------
train_loss 0.008376 val_loss 0.008495
learning rate:  1.0000000000000002e-07 , sim:  3
train_loss: 0.008098  [   64/250000]
train_loss: 0.006379  [32064/250000]
train_loss: 0.008225  [64064/250000]
train_loss: 0.008993  [96064/250000]
train_loss: 0.006837  [128064/250000]
train_loss: 0.009353  [160064/250000]
train_loss: 0.009722  [192064/250000]
train_loss: 0.008918  [224064/250000]
Epoch 241
-------------------------------
train_loss 0.008348 val_loss 0.008454
learning rate:  1.0000000000000002e-07 , sim:  3
Epoch 251
----------------------

Epoch 271
-------------------------------
train_loss 0.008368 val_loss 0.008454
learning rate:  1.0000000000000002e-07 , sim:  4
Epoch 281
-------------------------------
train_loss 0.008229 val_loss 0.008353
learning rate:  1.0000000000000002e-07 , sim:  4
Epoch 291
-------------------------------
train_loss 0.008377 val_loss 0.008482
learning rate:  1.0000000000000002e-07 , sim:  4
train_loss: 3.195873  [   64/250000]
train_loss: 1.273975  [32064/250000]
train_loss: 0.648411  [64064/250000]
train_loss: 0.237705  [96064/250000]
train_loss: 0.170905  [128064/250000]
train_loss: 0.066676  [160064/250000]
train_loss: 0.038077  [192064/250000]
train_loss: 0.025357  [224064/250000]
Epoch 1
-------------------------------
train_loss 0.018783 val_loss 0.018740
learning rate:  1e-05 , sim:  5
Epoch 11
-------------------------------
train_loss 0.009116 val_loss 0.009129
learning rate:  1e-05 , sim:  5
Epoch 21
-------------------------------
train_loss 0.008984 val_loss 0.009015
learning rate

...

Epoch 231
-------------------------------
train_loss 0.008306 val_loss 0.008372
learning rate:  1.0000000000000002e-07 , sim:  14
train_loss: 0.010523  [   64/250000]
train_loss: 0.008404  [32064/250000]
train_loss: 0.008692  [64064/250000]
train_loss: 0.008709  [96064/250000]
train_loss: 0.008012  [128064/250000]
train_loss: 0.009073  [160064/250000]
train_loss: 0.008785  [192064/250000]
train_loss: 0.007773  [224064/250000]
Epoch 241
-------------------------------
train_loss 0.008228 val_loss 0.008291
learning rate:  1.0000000000000002e-07 , sim:  14
Epoch 251
-------------------------------
train_loss 0.008560 val_loss 0.008639
learning rate:  1.0000000000000002e-07 , sim:  14
Epoch 261
-------------------------------
train_loss 0.008409 val_loss 0.008475
learning rate:  1.0000000000000002e-07 , sim:  14
train_loss: 0.011457  [   64/250000]
train_loss: 0.008437  [32064/250000]
train_loss: 0.008356  [64064/250000]
train_loss: 0.008027  [96064/250000]
train_loss: 0.008752  [128064/25

train_loss: 3.118960  [   64/250000]
train_loss: 1.090748  [32064/250000]
train_loss: 0.504350  [64064/250000]
train_loss: 0.347767  [96064/250000]
train_loss: 0.176564  [128064/250000]
train_loss: 0.096174  [160064/250000]
train_loss: 0.063333  [192064/250000]
train_loss: 0.033401  [224064/250000]
Epoch 1
-------------------------------
train_loss 0.023181 val_loss 0.023217
learning rate:  1e-05 , sim:  16
Epoch 11
-------------------------------
train_loss 0.008613 val_loss 0.008704
learning rate:  1e-05 , sim:  16
Epoch 21
-------------------------------
train_loss 0.008486 val_loss 0.008586
learning rate:  1e-05 , sim:  16
train_loss: 0.006105  [   64/250000]
train_loss: 0.007063  [32064/250000]
train_loss: 0.006667  [64064/250000]
train_loss: 0.007943  [96064/250000]
train_loss: 0.008297  [128064/250000]
train_loss: 0.009186  [160064/250000]
train_loss: 0.008878  [192064/250000]
train_loss: 0.007905  [224064/250000]
Epoch 31
-------------------------------
train_loss 0.008519 val_

train_loss: 0.009352  [192064/250000]
train_loss: 0.007915  [224064/250000]
Epoch 61
-------------------------------
train_loss 0.008642 val_loss 0.008730
learning rate:  1e-05 , sim:  17
Epoch 71
-------------------------------
train_loss 0.008427 val_loss 0.008528
learning rate:  1e-05 , sim:  17
Epoch 81
-------------------------------
train_loss 0.008566 val_loss 0.008652
learning rate:  1e-05 , sim:  17
train_loss: 0.008354  [   64/250000]
train_loss: 0.008299  [32064/250000]
train_loss: 0.007927  [64064/250000]
train_loss: 0.008327  [96064/250000]
train_loss: 0.007840  [128064/250000]
train_loss: 0.009679  [160064/250000]
train_loss: 0.009861  [192064/250000]
train_loss: 0.007818  [224064/250000]
Epoch 91
-------------------------------
train_loss 0.008498 val_loss 0.008607
learning rate:  1e-05 , sim:  17
Epoch 101
-------------------------------
train_loss 0.008292 val_loss 0.008402
learning rate:  1.0000000000000002e-06 , sim:  17
Epoch 111
-------------------------------
trai

Epoch 131
-------------------------------
train_loss 0.008709 val_loss 0.008829
learning rate:  1.0000000000000002e-06 , sim:  18
Epoch 141
-------------------------------
train_loss 0.008512 val_loss 0.008610
learning rate:  1.0000000000000002e-06 , sim:  18
train_loss: 0.007520  [   64/250000]
train_loss: 0.008327  [32064/250000]
train_loss: 0.006632  [64064/250000]
train_loss: 0.008086  [96064/250000]
train_loss: 0.008639  [128064/250000]
train_loss: 0.007410  [160064/250000]
train_loss: 0.008538  [192064/250000]
train_loss: 0.010730  [224064/250000]
Epoch 151
-------------------------------
train_loss 0.008306 val_loss 0.008415
learning rate:  1.0000000000000002e-06 , sim:  18
Epoch 161
-------------------------------
train_loss 0.008398 val_loss 0.008517
learning rate:  1.0000000000000002e-06 , sim:  18
Epoch 171
-------------------------------
train_loss 0.008472 val_loss 0.008608
learning rate:  1.0000000000000002e-06 , sim:  18
train_loss: 0.007560  [   64/250000]
train_loss: 0

Epoch 201
-------------------------------
train_loss 0.008337 val_loss 0.008468
learning rate:  1.0000000000000002e-07 , sim:  19
train_loss: 0.010715  [   64/250000]
train_loss: 0.009177  [32064/250000]
train_loss: 0.008848  [64064/250000]
train_loss: 0.006735  [96064/250000]
train_loss: 0.008389  [128064/250000]
train_loss: 0.009490  [160064/250000]
train_loss: 0.010074  [192064/250000]
train_loss: 0.009412  [224064/250000]
Epoch 211
-------------------------------
train_loss 0.008117 val_loss 0.008238
learning rate:  1.0000000000000002e-07 , sim:  19
Epoch 221
-------------------------------
train_loss 0.008152 val_loss 0.008286
learning rate:  1.0000000000000002e-07 , sim:  19
Epoch 231
-------------------------------
train_loss 0.008400 val_loss 0.008536
learning rate:  1.0000000000000002e-07 , sim:  19
train_loss: 0.007620  [   64/250000]
train_loss: 0.008293  [32064/250000]
train_loss: 0.007463  [64064/250000]
train_loss: 0.007960  [96064/250000]
train_loss: 0.006750  [128064/25